In [4]:
import yfinance as yf
import pandas as pd

# 1. Define the tickers and date range from the paper
tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN']
start_date = '2008-01-01'
end_date = '2009-12-31' # yfinance includes the end date

# 2. Download the data
# This downloads all tickers at once into a multi-index DataFrame
try:
    data = yf.download(tickers, start=start_date, end=end_date)
    
    if data.empty:
        raise Exception("No data downloaded. Check tickers, date range, and internet connection.")

    print("Data downloaded successfully. Inspecting columns...")
    # print(data.columns) # Uncomment this for deep debugging

    # 3. We only care about the 'Adj Close' price
    # 'Adj Close' is better than 'Close' as it accounts 
    # for dividends and stock splits.
    
    # When downloading multiple tickers, yfinance returns a MultiIndex
    # The top level is the measure ('Adj Close', 'Close', etc.)
    # The KeyError means 'Adj Close' was not found as a top-level column.
    
    # Check if 'Adj Close' is in the top level of the columns
    if 'Adj Close' not in data.columns.get_level_values(0):
        print("Warning: 'Adj Close' not found in downloaded data columns.")
        print("Available top-level columns:", data.columns.get_level_values(0).unique())
        
        # As a fallback, try to use 'Close'
        if 'Close' in data.columns.get_level_values(0):
            print("Falling back to using 'Close' price.")
            adj_close_data = data['Close']
        else:
            # If neither is present, we can't proceed
            raise Exception("'Adj Close' and 'Close' not found. Cannot proceed.")
    else:
        # This is the normal, expected path
        adj_close_data = data['Adj Close']
    
    # 4. Save to a CSV to use in your project
    adj_close_data.to_csv('stock_prices.csv')
    
    print("\nData processed and saved successfully!")
    print(adj_close_data.head())

except Exception as e:
    print(f"An error occurred: {e}")

C:\Users\percy\AppData\Local\Temp\ipykernel_35604\1788708486.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date)
[*********************100%***********************]  4 of 4 completed

Data downloaded successfully. Inspecting columns...
Available top-level columns: Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')
Falling back to using 'Close' price.

Data processed and saved successfully!
Ticker          AAPL    AMZN       GOOG       MSFT
Date                                              
2008-01-02  5.849120  4.8125  16.949791  25.299334
2008-01-03  5.851821  4.7605  16.953255  25.407078
2008-01-04  5.405123  4.4395  16.252443  24.695938
2008-01-07  5.332774  4.4410  16.060728  24.861151
2008-01-08  5.140944  4.3940  15.626096  24.027895


In [2]:
import numpy as np

# Compute log10(a_n / a_{n-1}) for all columns except the first
import numpy as np

log_returns = adj_close_data.copy()

# Compute ratios and take log10; this updates only the selected columns
log_returns = np.log10(log_returns / log_returns.shift(1))

# Drop the first row which will be NaN due to the shift and reset the index
log_returns = log_returns.dropna().reset_index(drop=True)

#"squash" using arctan
log_returns = np.arctan(log_returns)

# Show the first few transformed rows
print(log_returns.head())

Ticker      AAPL      AMZN      GOOG      MSFT
0       0.000201 -0.004718  0.000089  0.001846
1      -0.034472 -0.030309 -0.018332 -0.012328
2      -0.005852  0.000147 -0.005153  0.002896
3      -0.015909 -0.004621 -0.011914 -0.014804
4       0.020189 -0.013348  0.014548  0.012666


In [5]:
#Save squashed log returns

log_returns.to_csv('squashed_log_returns.csv', index=False)